In [20]:
import pygame,random,os
import statistics,math,time,copy
from enum import Enum
open_suitcase= pygame.transform.scale(pygame.image.load(os.path.join( 'open_suitcase.png')),(150,90))
closed_suitcase= pygame.transform.scale(pygame.image.load(os.path.join('closed_suitcase.png')),(150,90))

In [21]:
class hit_box():
    def __init__(self,_min_x,_max_x,_min_y,_max_y):
        self.min_x=_min_x
        self.max_x=_max_x
        self.min_y=_min_y
        self.max_y=_max_y
    def is_in_hitbox(self,x,y)->bool:
        if self.min_x>x or self.max_x<x:
            return False
        return self.min_y<=y and self.max_y>=y
    def get_rect(self):
        return (self.min_x,self.min_y,self.max_x-self.min_x,self.max_y-self.min_y)
class suitcase():
    def __init__(self,_amount,_number):
        self.amount=_amount
        self.number=_number
        self.is_open=False
        #dims=100x50
        x_pos=((((_number-1)%5))*150)+20
        y_pos=(math.floor((_number-1)/5)*125)+20
        self.hitbox=hit_box(x_pos,x_pos+100,y_pos,y_pos+50)
    def __repr__(self):
        if not self.is_open:
            return f"case #{self.number} isn't open"
        return f"case #{self.number} is worth ${self.amount:.2f}"
    def is_integer(self):
        return False
    @staticmethod
    def dealers_offer(suitcases):
        mean= statistics.mean([s.amount for s in suitcases if not s.is_open])*0.95
        if mean>=3:
            return math.floor(mean)
        return round(mean,2)
class amont_display():
    def render(self,_suitcases,_font,_screen):
       _suitcases=copy.deepcopy(_suitcases)
       _suitcases.sort(key=lambda x:x.amount)
       for i,case in enumerate(_suitcases):
            ts=_font.render(str(case.amount),True,
                            (230,20,10) if case.is_open else (255,255,255))
            _screen.blit(ts,(800,(50*i)+20))
class State(Enum):
    start=0
    delay_to_middle=1
    middle=2
    delay=3
    end=4
    realy_end=5
    lazy=6
def distr_gen(r):
    while r>1:
        if r in range(2):
            r-=1
            yield 1
        x=math.floor(r/3)
        if x==0:x=1
        r-=x
        yield x

suitcase_amount=[0.01,0.1,0.25,0.5,1,2,5,10,15,25,50]
rounds=[i for i in distr_gen(suitcase_amount.__len__())]
cases_left=rounds.pop(0)
random.shuffle(suitcase_amount)
suitcases=[suitcase(amount,i+1) for i,amount in enumerate(suitcase_amount)]
my_suitcase=-1

In [22]:
#init graphics
pygame.init()
pygame.font.init()
font=pygame.font.SysFont('Arial',30)
font2=pygame.font.SysFont('Arial',50)
WIDTH, HEIGHT = 1050, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("DEAL or NO DEAL")
running = True
state=State.start
delay_start=0
v=-1
l=amont_display()
while running:

    c=-1
    # Event handling
    if state==State.delay and delay_start+5e9>time.time_ns():state=State.middle
    for event in pygame.event.get():
        if event.type == pygame.QUIT:  # Check if the user clicked the close button
            running = False
        if event.type==pygame.MOUSEBUTTONDOWN and state!=State.realy_end:
            if state==State.delay:
                continue
            pos_x,pos_y=event.pos
            for i in suitcases:
                
                if state==State.end or state==State.realy_end:
                    if v==-1:
                        h=hit_box(20,120,400,500)
                        if h.is_in_hitbox(pos_x,pos_y):
                            state=State.realy_end
                            v=suitcases[my_suitcase].amount
                    for n in range(len(suitcases)):
                        if not i.is_open:
                            if v==-1:
                                v=suitcases[n].amount
                                state=State.realy_end
                            l=suitcases[n].amount
                        suitcases[n].is_open=True
                if not i.is_integer():
                    if i.hitbox.is_in_hitbox(pos_x,pos_y):
                        if state==State.start:
                            my_suitcase=i.number
                            state=State.delay
                            delay_start=time.time_ns()
                            break
                        print(i.number)
                        state=State.delay
                        delay_start=time.time_ns()
                        c=i.number-1
                        if state==State.end and v==-1:
                            v=i.amount
    if state==State.lazy:continue
    if c!=-1:
        if not suitcases[c].is_open:
            if cases_left==0:
                cases_left=rounds.pop(0)
                if len(rounds)==1:
                    state=State.end
            cases_left-=1
        suitcases[c].is_open=True

    screen.fill((0, 0, 0)) 
    for i in suitcases:
        if i.number==my_suitcase:continue
        if not i.is_open:
            screen.blit(closed_suitcase, i.hitbox.get_rect())
        else:
            screen.blit(open_suitcase, i.hitbox.get_rect())
        text= str(i.number) if not i.is_open else "$"+str(i.amount)
        ts=font.render(text,True,(0,0,0))
        screen.blit(ts,(i.hitbox.get_rect()[0]+60,i.hitbox.get_rect()[1]+30))
    if state==State.middle or state==State.delay or state.end and my_suitcase!=-1:
        screen.blit(closed_suitcase,(20,400,100,50))
        text=str(my_suitcase)
        ts=font.render(text,True,(0,0,0))
        screen.blit(ts,(90,425))
    if state==State.start:
        ts=font2.render(f"Pick a case",True,(255,255,255))
        screen.blit(ts,(150,500))
    elif state==state.end:
        ts=font2.render(f"Keep your case or change it?",True,(255,255,255))
        screen.blit(ts,(150,500))
    elif state==State.realy_end:
        pygame.draw.rect(screen,(0,0,120),(20,400,100,50))
        text="$"+str(suitcases[my_suitcase].amount)
        ts=font.render(text,True,(0,0,0))
        screen.blit(ts,(60,405))
        if v==-1:
            state=State.lazy
        ts=font2.render(f"You won ${v} dollars",True,(255,255,255))
        screen.blit(ts,(150,500))
    elif cases_left!=0:
        ts=font2.render(f"Open {cases_left}  until banker offer",True,(255,255,255))
        screen.blit(ts,(150,500))
    else:
        try:
            ts=font2.render(f"Banker's offer ${suitcase.dealers_offer(suitcases)}",True,(255,255,255))
            screen.blit(ts,(150,440))
        except:
            state=State.realy_end
            continue
        if len(rounds)==0:
            state=State.realy_end
            continue
        if state!=State.end:
            ts=font2.render(f"Next round open {rounds[0]} suitcases",True,(255,255,255))
            screen.blit(ts,(150,500))
    l.render(suitcases,font,screen)
    # Update the display
    pygame.display.flip()  # or pygame.display.update()

# 5. Quit Pygame
pygame.quit()

9
10


In [23]:
while False:
    print("here are your suitcases:")
    print(suitcases)
    try:
        choice=int(input("Pick a suitcase from 1-12 "))-1
    except:
        print("NOOOOOOOOOOOOOOOOOOOOOOOOOO!!!!!!!!!!!!!!!")
        continue
    if choice in range(12):
        if suitcases[choice].is_open:
            print("THAT SUITCASE IS OPEN")
            continue
        if choice==P_suitcase:
            print("THATS YOUR SUITCASE")
            continue
    else:
        print("WHAAAAAAAAT ARE YOU THINKING????????????????????")
        continue
    suitcases[choice].is_open=True
    print(f"That suitcase had ${suitcases[choice]}")
    print(f"Banker's offer: ${suitcase.dealers_offer(suitcases):.2f}")
    a=input("DEAL or NO DEAL? ").lower()
    if a=="deal":
        break
